In [1]:
from glob import glob
from datetime import datetime, timezone

import netCDF4
import matplotlib.pyplot as plt

import pyart

import read_mira_radar


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [6]:
# test = pyart.aux_io.read_kazr("/home/camila/git/amazon-storms-aerosols/radar_processing/mira_quicklooks/20200508_1200.nc")

test = netCDF4.Dataset("/home/camila/git/amazon-storms-aerosols/radar_processing/mira_quicklooks/20200508_1200.nc")
test

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    convention: CF-1.0
    location: ATTO Campina
    system: C5XC
    title: MIRA Cloud Radar Data
    institution: USP
    source: 200508_120004.pds.off
    reference: Ka Band Cloud Radar MIRA, METEK GmbH www.metek.de
    Copywright: 
    Copywright_Owner: 
    notes: It is accounted for the Receiver Bwd Loss in Radarconst*, and Z*.
    dimensions(sizes): time(351), range(573)
    variables(dimensions): float32 filtered_reflectivity(range, time), float32 filtered_velocity(range, time), float32 filtered_linear_depolarization_ratio(range, time), float32 cross_correlation_ratio(range, time), float32 differential_phase(range, time), float32 nyquist_velocity(), int32 n_fft(), int32 prf(), float64 prt(), int32 n_range_gates(), float32 range_resolution(), int32 n_samples(), float64 pulse_width(), float64 frequency(), float64 latitude(), float64 longitude(), float64 altitude(), float32 range(range), int64 t

In [ ]:
data_path = "/home/camila/git/amazon-storms-aerosols/data/radar/mira_campina/"

# Single file
# pmira, pmira_melthei, pmira_mrm = read_mira_radar.read_mira(data_path + "2021/11/09/20211109_0900.mmclx")
# mira1 = netCDF4.Dataset(data_path + "2021/11/09/20211109_0900.mmclx")
# mira2 = netCDF4.Dataset(data_path + "2021/09/28/20210928_0900.mmclx")

# Multiple files
# files = sorted(glob(data_path + "2021/11/09/" + "*.mmclx", recursive=True))
# files = sorted(glob(data_path + "test_files/" + "*_rp.mmclx", recursive=True))
# pmira_24h, pmira_melthei_24h, pmira_mrm_24h = read_mira_radar.read_multi_mira(
#     files, for_quicklooks=True, ql_res=1
# )
files = []
files.append(sorted(glob(data_path + "2021/06/11/" + "*.mmclx", recursive=True)))
files.append(sorted(glob(data_path + "2021/06/12/" + "*.mmclx", recursive=True)))
files.append(sorted(glob(data_path + "2021/06/13/" + "*.mmclx", recursive=True)))
files.append(sorted(glob(data_path + "2021/06/14/" + "*.mmclx", recursive=True)))
files.append(sorted(glob(data_path + "2021/06/15/" + "*.mmclx", recursive=True)))
files.append(sorted(glob(data_path + "2021/06/16/" + "*.mmclx", recursive=True)))
files.append(sorted(glob(data_path + "2021/06/17/" + "*.mmclx", recursive=True)))
files = [item for sublist in files for item in sublist]
# print(files)
pmira_w, pmira_melthei_w, pmira_mrm_w = read_mira_radar.read_multi_mira(
    files, for_quicklooks=True, ql_res=30
)

In [ ]:
for var in mira.variables.keys():
    print(var, "-", mira.variables[var].long_name)

# print(mira.fields.keys())
# print(mira.variables["npw1"])

In [ ]:
times = pyart.util.datetimes_from_radar(pmira_24h)
times = times.astype("datetime64[ns]")
date_str = str(times[0])[:10]

# Health
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "SNRg",
        vmin=pmira_24h.fields["SNRg"]["yrange"][0],
        vmax=pmira_24h.fields["SNRg"]["yrange"][1],
        cmap="pyart_Carbone17",
        title="Health status - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
plt.plot(times, 5 + pmira_mrm_24h["data"]*1e1, "k-", label="Radiometric noise power")
plt.axhline(y=15, color="k", linestyle="dashed")
display.plot_grid_lines()
plt.ylim((0, 18))
plt.legend(loc="upper right")

# Reflectivity
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "Ze",
        vmin=pmira_24h.fields["Ze"]["yrange"][0],
        vmax=pmira_24h.fields["Ze"]["yrange"][1],
        cmap="pyart_Carbone17",
        title="Reflectivity (filtered) - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
display.plot_grid_lines()
plt.ylim((0, 18))

# Vel_Doppler
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "VEL",
        vmin=pmira_24h.fields["VEL"]["yrange"][0],
        vmax=pmira_24h.fields["VEL"]["yrange"][1],
        cmap="pyart_BuDRd18",
        title="Doppler Velocity (filtered) - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
plt.ylim((0, 18))
display.plot_grid_lines()

# LDR
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "LDRg",
        vmin=pmira_24h.fields["LDRg"]["yrange"][0],
        vmax=pmira_24h.fields["LDRg"]["yrange"][1],
        cmap="pyart_SCook18",
        title="Linear De-Polarization Ratio (unfiltered) - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
plt.ylim((0, 18))
display.plot_grid_lines()


In [ ]:
times = pyart.util.datetimes_from_radar(pmira_24h)
times = times.astype("datetime64[ns]")
date_str = str(times[0])[:10]

# Health
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "SNRg",
        vmin=pmira_24h.fields["SNRg"]["yrange"][0],
        vmax=pmira_24h.fields["SNRg"]["yrange"][1],
        cmap="pyart_Carbone17",
        title="Health status - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
plt.plot(times, 5 + pmira_mrm_24h["data"]*1e1, "k-", label="Radiometric noise power")
plt.axhline(y=15, color="k", linestyle="dashed")
display.plot_grid_lines()
plt.ylim((0, 18))
plt.legend(loc="upper right")

# Reflectivity
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "Ze",
        vmin=pmira_24h.fields["Ze"]["yrange"][0],
        vmax=pmira_24h.fields["Ze"]["yrange"][1],
        cmap="pyart_Carbone17",
        title="Reflectivity (filtered) - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
display.plot_grid_lines()
plt.ylim((0, 18))

# Vel_Doppler
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "VEL",
        vmin=pmira_24h.fields["VEL"]["yrange"][0],
        vmax=pmira_24h.fields["VEL"]["yrange"][1],
        cmap="pyart_BuDRd18",
        title="Doppler Velocity (filtered) - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
plt.ylim((0, 18))
display.plot_grid_lines()

# LDR
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "LDRg",
        vmin=pmira_24h.fields["LDRg"]["yrange"][0],
        vmax=pmira_24h.fields["LDRg"]["yrange"][1],
        cmap="pyart_SCook18",
        title="Linear De-Polarization Ratio (unfiltered) - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
plt.ylim((0, 18))
display.plot_grid_lines()


In [ ]:
times = pyart.util.datetimes_from_radar(pmira_24h)
times = times.astype("datetime64[ns]")
date_str = str(times[0])[:10]

# Health
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "SNRg",
        vmin=pmira_24h.fields["SNRg"]["yrange"][0],
        vmax=pmira_24h.fields["SNRg"]["yrange"][1],
        cmap="pyart_Carbone17",
        title="Health status - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
plt.plot(times, 5 + pmira_mrm_24h["data"]*1e1, "k-", label="Radiometric noise power")
plt.axhline(y=15, color="k", linestyle="dashed")
display.plot_grid_lines()
plt.ylim((0, 18))
plt.legend(loc="upper right")
plt.savefig("figs/Mira35_Health_Campina_" + date_str.replace("-", "_") + ".png", dpi=300, bbox_inches="tight")

# Reflectivity
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "Ze",
        vmin=pmira_24h.fields["Ze"]["yrange"][0],
        vmax=pmira_24h.fields["Ze"]["yrange"][1],
        cmap="pyart_Carbone17",
        title="Reflectivity (filtered) - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
display.plot_grid_lines()
plt.ylim((0, 18))
plt.savefig("figs/Mira35_Reflectivity_Campina_" + date_str.replace("-", "_") + ".png", dpi=300, bbox_inches="tight")

# Vel_Doppler
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "VEL",
        vmin=pmira_24h.fields["VEL"]["yrange"][0],
        vmax=pmira_24h.fields["VEL"]["yrange"][1],
        cmap="pyart_BuDRd18",
        title="Doppler Velocity (filtered) - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
plt.ylim((0, 18))
display.plot_grid_lines()
plt.savefig("figs/Mira35_Vel_Doppler_Campina_" + date_str.replace("-", "_") + ".png", dpi=300, bbox_inches="tight")

# LDR
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_24h)
display.plot_vpt(
        "LDRg",
        vmin=pmira_24h.fields["LDRg"]["yrange"][0],
        vmax=pmira_24h.fields["LDRg"]["yrange"][1],
        cmap="pyart_SCook18",
        title="Linear De-Polarization Ratio (unfiltered) - " + date_str,
        time_axis_flag=True,
        mask_outside=True,
        raster=True,
    )
plt.ylim((0, 18))
display.plot_grid_lines()
plt.savefig("figs/Mira35_LDR_Campina_" + date_str.replace("-", "_") + ".png", dpi=300, bbox_inches="tight")

In [ ]:
pmira_24h.fields["Ze"]

In [ ]:
times = pyart.util.datetimes_from_radar(pmira_w)
date_str = times[len(times)%2]._to_real_datetime().date().isocalendar()
date_str = str(date_str[0]) + " week " + str(date_str[1])
# times = times.astype("datetime64[ns]")

# Health
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_w)
display.plot_vpt(
        "SNRg",
        vmin=pmira_w.fields["SNRg"]["yrange"][0],
        vmax=pmira_w.fields["SNRg"]["yrange"][1],
        cmap="pyart_Carbone17",
        title="Health status - " + date_str,
        time_axis_flag=True,
        date_time_form="%Y-%m-%d %H:%M",
        axislabels=(("", None)),
        mask_outside=True,
        raster=True,
    )
plt.plot(times, 5 + pmira_mrm_w["data"]*1e1, "k-", label="Radiometric noise power")
plt.axhline(y=15, color="k", linestyle="dashed")
display.plot_grid_lines()
plt.ylim((0, 18))
plt.legend(loc="upper right")
plt.savefig("figs/Mira35_Health_Campina_" + date_str.replace(" ", "_") + ".png", dpi=300, bbox_inches="tight")

# Reflectivity
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_w)
display.plot_vpt(
        "Ze",
        vmin=pmira_w.fields["Ze"]["yrange"][0],
        vmax=pmira_w.fields["Ze"]["yrange"][1],
        cmap="pyart_Carbone17",
        title="Reflectivity (Filtered) - " + date_str,
        time_axis_flag=True,
        date_time_form="%Y-%m-%d %H:%M",
        axislabels=(("", None)),
        mask_outside=True,
        raster=True,
    )
display.plot_grid_lines()
plt.ylim((0, 18))
plt.savefig("figs/Mira35_Reflectivity_Campina_" + date_str.replace(" ", "_") + ".png", dpi=300, bbox_inches="tight")

# Vel_Doppler
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_w)
display.plot_vpt(
        "VEL",
        vmin=pmira_w.fields["VEL"]["yrange"][0],
        vmax=pmira_w.fields["VEL"]["yrange"][1],
        cmap="pyart_BuDRd18",
        title="Doppler Velocity (filtered) - " + date_str,
        time_axis_flag=True,
        date_time_form="%Y-%m-%d %H:%M",
        axislabels=(("", None)),
        mask_outside=True,
        raster=True,
    )
plt.ylim((0, 18))
display.plot_grid_lines()
plt.savefig("figs/Mira35_Vel_Doppler_Campina_" + date_str.replace(" ", "_") + ".png", dpi=300, bbox_inches="tight")

# LDR
fig = plt.figure(figsize=(15, 5))
display = pyart.graph.RadarDisplay(pmira_w)
display.plot_vpt(
        "LDRg",
        vmin=pmira_w.fields["LDRg"]["yrange"][0],
        vmax=pmira_w.fields["LDRg"]["yrange"][1],
        cmap="pyart_SCook18",
        title="Linear De-Polarization Ratio (unfiltered) - " + date_str,
        time_axis_flag=True,
        date_time_form="%Y-%m-%d %H:%M",
        axislabels=(("", None)),
        mask_outside=True,
        raster=True,
    )
plt.ylim((0, 18))
display.plot_grid_lines()
plt.savefig("figs/Mira35_LDR_Campina_" + date_str.replace(" ", "_") + ".png", dpi=300, bbox_inches="tight")

In [ ]:
plt.pcolormesh(mira.variables["Zg"])

In [ ]:
# mrm = (mira1.variables["MRMcx"][:] + mira1.variables["MRMco"][:])/2 - 1.0942738
# plt.plot(mrm)
# plt.hlines(xmin=0, xmax=703, y=0)
# plt.ylim(0,1)
plt.plot(mira1.variables["MRMcx"])
# plt.plot(mira1.variables["MRMco"])

In [ ]:
mrm = (mira2.variables["MRMco"][:] + mira2.variables["MRMco"][:])/2 - 1.0942738
plt.plot(mrm)
plt.hlines(xmin=0, xmax=703, y=0)
plt.ylim(0,0.2)
# plt.plot(mira.variables["MRMcx"])
# plt.plot(mira.variables["MRMco"])
